In [2]:
# ===============================================================
# YourDOST SDE Intern Assignment – Backend (Flask + JSON)
# Author: Shubham Mahind
# ===============================================================

!pip install flask-ngrok --quiet

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import json, os

# ---------- Setup ----------
DATA_FILE = "/content/todos.json"   # explicit absolute path for Colab
app = Flask(__name__)
run_with_ngrok(app)


# ---------- Helper Functions ----------
def read_todos():
    """Read all todos from the JSON file (create if missing)."""
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, "w") as f:
            json.dump([], f)
    with open(DATA_FILE, "r") as f:
        return json.load(f)

def write_todos(todos):
    """Write todos list back to JSON file."""
    with open(DATA_FILE, "w") as f:
        json.dump(todos, f, indent=2)


# ---------- Routes ----------
@app.route("/todos", methods=["GET"])
def get_todos():
    return jsonify(read_todos()), 200


@app.route("/todos", methods=["POST"])
def create_todo():
    data = request.get_json()
    if not data or "title" not in data:
        return jsonify({"error": "Title is required"}), 400

    todos = read_todos()
    new_todo = {
        "id": len(todos) + 1,
        "title": data["title"],
        "completed": data.get("completed", False)
    }
    todos.append(new_todo)
    write_todos(todos)
    return jsonify(new_todo), 201


@app.route("/todos/<int:todo_id>", methods=["PUT"])
def update_todo(todo_id):
    todos = read_todos()
    for todo in todos:
        if todo["id"] == todo_id:
            data = request.get_json()
            todo["title"] = data.get("title", todo["title"])
            todo["completed"] = data.get("completed", todo["completed"])
            write_todos(todos)
            return jsonify(todo), 200
    return jsonify({"error": "Todo not found"}), 404


@app.route("/todos/<int:todo_id>", methods=["DELETE"])
def delete_todo(todo_id):
    todos = read_todos()
    updated = [t for t in todos if t["id"] != todo_id]
    if len(updated) == len(todos):
        return jsonify({"error": "Todo not found"}), 404
    write_todos(updated)
    return jsonify({"message": "Todo deleted"}), 200


@app.route("/status", methods=["GET"])
def status():
    return jsonify({"status": "Server running successfully ✅"}), 200


# ---------- Run ----------
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            